In [ ]:
# Installing conda
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0   7316      0 --:--:-- --:--:-- --:--:--  7316


all packages are already installed


# conda environments:
#
base                  *  /root/miniconda



In [ ]:
# Installing Deepchem
!pip install --pre deepchem
# Import deepchem just to check the version
import deepchem
deepchem.__version__

'2.4.0-rc1.dev'

In [5]:
transformers

In [17]:
class_pred_ovr

NameError: name 'class_pred_ovr' is not defined

In [24]:
[x.X.shape for x in tox21_datasets]

[(6264, 1024), (783, 1024), (784, 1024)]

In [18]:
datasets = {'train': {'X': train_dataset.X, 'y': train_dataset.y,},
           'valid': {'X': valid_dataset.X, 'y': valid_dataset.y}} 

In [19]:
with open('../test_data.pickle', 'wb') as f:
    pickle.dump(datasets, f)

In [34]:
datasets['train']['y'].shape

(6264, 12)

In [20]:
datasets['train']['y'][0].shape

(12,)

In [4]:
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import numpy as np
import deepchem as dc
import tempfile
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Only for debug!
np.random.seed(123)

# Load Tox21 dataset
n_features = 1024
#import pdb
#pdb.set_trace()
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()
train_dataset, valid_dataset, test_dataset = tox21_datasets

#train_dataset.y = train_dataset.y.astype('int')
#valid_dataset.y = valid_dataset.y.astype('int')
#test_dataset.y = test_dataset.y.astype('int')

# Fit models
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)


def model_builder(model_dir):
  sklearn_model = SVC(C=1.0, class_weight="balanced", probability=True)
  return dc.models.SklearnModel(sklearn_model, model_dir)


model_dir = tempfile.mkdtemp()
model = dc.models.SingletaskToMultitask(tox21_tasks, model_builder, model_dir)

forest_model = clf = RandomForestClassifier(n_estimators=400, max_depth=100)
forest_model.fit(train_dataset.X, train_dataset.y.astype('int'))

class_pred_rf = forest_model.predict(valid_dataset.X)


ovrclf = OneVsRestClassifier(LinearSVC(random_state=0))
ovrclf.fit(train_dataset.X, train_dataset.y.astype('int'))

class_pred_ovr = ovrclf.predict(valid_dataset.X)


# Fit trained model
#model.fit(train_dataset)
#model.save()

#print("Evaluating model")
#train_scores = model.evaluate(train_dataset, [metric], transformers)
#valid_scores = model.evaluate(valid_dataset, [metric], transformers)

#print("Train scores")
#print(train_scores)

#print("Validation scores")
#print(valid_scores)

/home/ryan/miniconda3/envs/dcenv_test4/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/home/ryan/miniconda3/envs/dcenv_test4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ryan/miniconda3/envs/dcenv_test4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version 

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.
About to initialize singletask to multitask model
Initializing directory for task NR-AR
Initializing directory for task NR-AR-LBD
Initializing directory for task NR-AhR
Initializing directory for task NR-Aromatase
Initializing directory for task NR-ER
Initializing directory for task NR-ER-LBD
Initializing directory for task NR-PPAR-gamma
Initializing directory for task SR-ARE
Initializing directory for task SR-ATAD5
Initializing directory for task SR-HSE
Initializing directory for task SR-MMP
Initializing directory for task SR-p53


/home/ryan/miniconda3/envs/dcenv_test4/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/ryan/miniconda3/envs/dcenv_test4/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/ryan/miniconda3/envs/dcenv_test4/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/ryan/miniconda3/envs/dcenv_test4/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/ryan/miniconda3/envs/dcenv_test4/lib/python3.6/site-packages/sklearn/svm/_base

In [5]:
from sklearn.metrics import accuracy_score
accuracy_score(valid_dataset.y, class_pred_ovr)

0.5006385696040868

In [6]:
accuracy_score(valid_dataset.y, class_pred_rf)

0.6372924648786717

In [7]:
import pickle
with open('../models/2020-11-15-RF-model.pickle', 'wb') as f:
    pickle.dump(forest_model, f)
    

In [13]:
forest_model.predict(valid_dataset.X).shape

(783, 12)

In [14]:
accuracy_score(valid_dataset.y, forest_model.predict(valid_dataset.X))

0.6372924648786717

In [1]:
import sys
sys.path.insert(0,'../')
from src.model import predict

In [17]:
np.all(predict(valid_dataset.X, forest_model) == class_pred_rf)

True

In [2]:
import pickle

# load model
with open('../models/2020-11-15-RF-model.pickle', 'rb') as f:
    model = pickle.load(f)

# load data
with open('../test_data.pickle', 'rb') as f:
    data = pickle.load(f)

Xval = data['valid']['X']
yval = data['valid']['y']

y_pred = predict(Xval, model)
#from sklearn.metrics import accuracy_score
#print(accuracy_score(yval, y_pred))

In [5]:
import src

In [6]:
dir(src)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'predict']

In [4]:
src.labels.inverse_transform([[0,1,0,1,0,1,0,1,0,1,0,1]])[0]

['estrogen receptor alpha, LBD (ER, LBD): inactive',
 'estrogen receptor alpha, full (ER, full): active',
 'aromatase: inactive',
 'aryl hydrocarbon receptor (AhR): active',
 'androgen receptor, full (AR, full): inactive',
 'androgen receptor, LBD (AR, LBD): active',
 'peroxisome proliferator-activated receptor gamma (PPAR-gamma): inactive',
 'nuclear factor (erythroid-derived 2)-like 2/antioxidant responsive element (Nrf2/ARE): active',
 'heat shock factor response element (HSE): inactive',
 'ATAD5: active',
 'mitochondrial membrane potential (MMP): inactive',
 'p53: active']